<a href="https://colab.research.google.com/github/bilmark0/Agile-Manufacturing-TDK-/blob/main/CV/Vision_Transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wandb

In [ ]:
import os
import shutil
import random
from google.colab import drive
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback
import PIL
import PIL.Image
import sys
from google.colab import files
import math

In [ ]:
# Upload the kaggle.json file
uploaded = files.upload()

# Move the uploaded file to the .kaggle directory
kaggle_api_path = os.path.expanduser("~/.kaggle")
if not os.path.exists(kaggle_api_path):
    os.makedirs(kaggle_api_path)

# Ensure the file is set with proper permissions
kaggle_json_path = next(iter(uploaded))  # Get the uploaded filename
os.rename(kaggle_json_path, f"{kaggle_api_path}/kaggle.json")
os.chmod(f"{kaggle_api_path}/kaggle.json", 0o600)

# Download the dataset from Kaggle
!kaggle datasets download -d markbilszky/agile-manufacturing-tdk --unzip

print("Dataset downloaded successfully.")

In [ ]:
# Set your paths here
base_path = './'  # Replace with the directory containing Reference, error_2, and error_3
new_folder_path = os.path.join(base_path, 'training_data')

# Create the new folder
os.makedirs(new_folder_path, exist_ok=True)

# Move the folders
folders_to_move = ['Reference', 'error_2', 'error_3']
for folder_name in folders_to_move:
    shutil.move(os.path.join(base_path, folder_name), new_folder_path)

print("Folders moved successfully!")

In [ ]:
import math
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf

# Sinusoidal learning rate callback
class SinusoidalLearningRate(callbacks.Callback):
    def __init__(self, max_lr, min_lr, total_epochs):
        super(SinusoidalLearningRate, self).__init__()
        self.max_lr = max_lr
        self.min_lr = min_lr
        self.total_epochs = total_epochs

    def on_epoch_begin(self, epoch, logs=None):
        lr = self.min_lr + (self.max_lr - self.min_lr) * (0.5 * (1 + math.sin(2 * math.pi * epoch / self.total_epochs)))
        self.model.optimizer.learning_rate.assign(lr)
        print(f"Epoch {epoch + 1}: Learning rate is {lr:.6f}")

# Hyperparameters for the sinusoidal learning rate
max_lr = 0.002  # Slightly increased max_lr
min_lr = 0.0001
total_epochs = 1000

# Use the sinusoidal learning rate in the callbacks
sinusoidal_lr_callback = SinusoidalLearningRate(max_lr=max_lr, min_lr=min_lr, total_epochs=total_epochs)

data_dir = '/path/to/your/faulty_directory'
img_height, img_width = 240, 380

# Enhanced data generator with more augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # New data augmentation: rotation
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2],  # New augmentation: brightness adjustment
    validation_split=0.1
)

# Training and validation data generators
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',  # Updated for multi-class classification
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Define the Vision Transformer model
def create_vit_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(16, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer='l2')(inputs)  # Added L2 regularization
    x = layers.MaxPooling2D((2, 2))(x)

    # Reshape for transformer input
    x = layers.Reshape((-1, x.shape[-1]))(x)

    # Transformer block
    for _ in range(4):  # Number of transformer blocks
        x_res = x
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        attention_output = layers.MultiHeadAttention(num_heads=4, key_dim=16)(x, x)  # Updated key_dim for complexity
        x = layers.Add()([x_res, attention_output])
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        x_res = x
        x = layers.Dense(128, activation='relu', kernel_regularizer='l2')(x)  # Enhanced dense layer
        x = layers.Dropout(0.3)(x)  # Added dropout within transformer block
        x = layers.Dense(x.shape[-1])(x)
        x = layers.Add()([x_res, x])

    x = layers.GlobalAveragePooling1D()(x)  # Changed to GlobalAveragePooling1D for consistency
    x = layers.Dense(512, activation='relu', kernel_regularizer='l2')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(3, activation='softmax')(x)  # Updated output for 3-class classification

    model = models.Model(inputs, outputs)
    return model

# Create and compile the model
model = create_vit_model((img_height, img_width, 3))
model.compile(optimizer=Adam(learning_rate=max_lr),
              loss='categorical_crossentropy',  # Updated loss for multi-class classification
              metrics=['accuracy'])

# Set up early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1)

# Train the model with the sinusoidal learning rate callback
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=total_epochs,
    callbacks=[early_stopping, model_checkpoint, sinusoidal_lr_callback, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')
